In [1]:
pip install pillow pillow-heif


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 12.0 MB/s  0:00:00 eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [7]:

import os
import io
import cv2
import numpy as np
from PIL import Image
from pillow_heif import register_heif_opener

register_heif_opener()

INPUT_DIR = "photos"     # 原图文件夹
OUTPUT_DIR = "public/photos"   # 输出文件夹

TARGET_RATIO = 4 / 3
OUT_W, OUT_H = 1200, 900

MAX_KB = 200

Q_MIN = 30
Q_MAX = 95
ITER = 9
# ========================

os.makedirs(OUTPUT_DIR, exist_ok=True)

EXTS = (".jpg", ".jpeg", ".png", ".heic", ".HEIC")

files = sorted(
    f for f in os.listdir(INPUT_DIR)
    if f.lower().endswith(EXTS)
)

face_cascade = cv2.CascadeClassifier(
    cv2.data.haarcascades + "haarcascade_frontalface_default.xml"
)

def detect_face_center(img_np):
    gray = cv2.cvtColor(img_np, cv2.COLOR_RGB2GRAY)
    faces = face_cascade.detectMultiScale(
        gray, scaleFactor=1.1, minNeighbors=5
    )
    if len(faces) == 0:
        return None
    x, y, w, h = max(faces, key=lambda f: f[2] * f[3])
    return x + w // 2, y + h // 2


def crop_around(img, cx, cy):
    w, h = img.size
    if w / h > TARGET_RATIO:
        crop_h = h
        crop_w = int(h * TARGET_RATIO)
    else:
        crop_w = w
        crop_h = int(w / TARGET_RATIO)

    left = int(cx - crop_w / 2)
    top = int(cy - crop_h / 2)

    left = max(0, min(left, w - crop_w))
    top = max(0, min(top, h - crop_h))

    return img.crop((left, top, left + crop_w, top + crop_h))


def jpeg_under_max(img):
    best = None
    low, high = Q_MIN, Q_MAX

    for _ in range(ITER):
        q = (low + high) // 2
        buf = io.BytesIO()
        img.save(buf, "JPEG", quality=q, optimize=True, progressive=True)
        size_kb = buf.tell() / 1024

        if size_kb <= MAX_KB:
            best = buf.getvalue()
            low = q + 1  # 尝试更高质量
        else:
            high = q - 1

    return best


index = 1

for name in files:
    src = os.path.join(INPUT_DIR, name)

    try:
        with Image.open(src) as img:
            img = img.convert("RGB")
            np_img = np.array(img)

            center = detect_face_center(np_img)
            if center:
                cropped = crop_around(img, *center)
            else:
                w, h = img.size
                cropped = crop_around(img, w // 2, h // 2)

            cropped = cropped.resize((OUT_W, OUT_H), Image.LANCZOS)

            jpeg_bytes = jpeg_under_max(cropped)

            out = os.path.join(OUTPUT_DIR, f"{index}.jpg")
            with open(out, "wb") as f:
                f.write(jpeg_bytes)

            size_kb = os.path.getsize(out) / 1024
            print(f"✓ {name} → {index}.jpg ({size_kb:.1f} KB)")
            index += 1

    except Exception as e:
        print(f"✗ 跳过 {name}: {e}")

print("✅ 全部处理完成")

✓ 04df2df5d6dea6676e8fff3230e8fbc4.JPG → 1.jpg (186.3 KB)
✓ 24124_livephoto.HEIC → 2.jpg (198.0 KB)
✓ 24125_livephoto.HEIC → 3.jpg (189.2 KB)
✓ 24550_livephoto.HEIC → 4.jpg (198.0 KB)
✓ 41b7436a34b044a0861cd85ff995e829.JPG → 5.jpg (181.7 KB)
✓ 57e88abb2161d165f4231dc216b4eba6.JPG → 6.jpg (187.5 KB)
✓ 5a331ef4d87b41c59ad0c3a9a18a71da.JPG → 7.jpg (197.8 KB)
✓ 721cf72a4988a360564277c03736c0e8.JPG → 8.jpg (197.8 KB)
✓ 875602d161a92e9898bcd79ad867c30c.HEIC → 9.jpg (193.8 KB)
✓ 9e2404849404a953655c597acc63d45c.JPG → 10.jpg (196.7 KB)
✓ IMG_3589.JPG → 11.jpg (197.5 KB)
✓ IMG_4059.HEIC → 12.jpg (198.6 KB)
✓ IMG_4288.HEIC → 13.jpg (196.6 KB)
✓ IMG_4468.HEIC → 14.jpg (199.2 KB)
✓ IMG_4619.HEIC → 15.jpg (199.9 KB)
✓ IMG_6925.HEIC → 16.jpg (192.5 KB)
✓ IMG_6926.HEIC → 17.jpg (192.7 KB)
✓ IMG_6956.HEIC → 18.jpg (195.3 KB)
✓ IMG_6957.HEIC → 19.jpg (192.8 KB)
✓ IMG_7265.HEIC → 20.jpg (198.0 KB)
✓ IMG_7315.HEIC → 21.jpg (190.0 KB)
✓ IMG_7507.HEIC → 22.jpg (195.8 KB)
✓ IMG_7532.HEIC → 23.jpg (198.2 KB)